In [1]:
from zipline import run_algorithm

from zipline.api import order_target_percent, symbol

from datetime import datetime

import pytz

ModuleNotFoundError: No module named 'zipline'

In [ ]:
%matplotlib inline

import numpy as np

import pandas as pd

from scipy import stats

import seaborn as sn

import matplotlib.pyplot as plt

import mpld3

mpld3.enable_notebook()

import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 2)

In [ ]:
merged_df = A.join(B, how='outer', lsuffix='_A', sort=True).join(C, how='outer', lsuffix='_B', rsuffix='_C', sort=True)

merged_df

In [ ]:
merged_df[merged_df.isnull().all(axis=1)]

In [ ]:
merged_df[['Close_A', 'Close_B', 'Close_C']].isnull().any(axis=1).sum()

In [ ]:
valid_close_df = merged_df.dropna(subset=['Close_A', 'Close_B', 'Close_C'], how='any')

In [ ]:
valid_close_df.isnull().any(axis=1).sum()

In [ ]:
valid_close_df[valid_close_df.isnull().any(axis=1)]

In [ ]:
valid_close_complete = valid_close_df.fillna(method='backfill')

In [ ]:
valid_close_complete.isnull().any(axis=1).sum()

In [ ]:
valid_close_complete['Open_A'].plot(figsize=(12,6), linestyle='--', color='black', legend='Open_A')

valid_close_complete['Close_A'].plot(figsize=(12,6), linestyle='-', color='grey', legend='Close_A')

valid_close_complete['Low_A'].plot(figsize=(12,6), linestyle=':', color='black', legend='Low_A')

valid_close_complete['High_A'].plot(figsize=(12,6), linestyle='-.', color='grey', legend='High_A')

In [ ]:
valid_close_complete['Open_C'].plot(figsize=(12,6), linestyle='--', color='black', legend='Open_C')

valid_close_complete['Close_C'].plot(figsize=(12,6), linestyle='-', color='grey', legend='Close_C')

valid_close_complete['Low_C'].plot(figsize=(12,6), linestyle=':', color='black', legend='Low_C')

valid_close_complete['High_C'].plot(figsize=(12,6), linestyle='-.', color='grey', legend='High_C')

In [ ]:
no_outlier_prices = prices_only[(np.abs(stats.zscore(prices_only)) < 6).all(axis=1)]

In [ ]:
no_outlier_prices['Open_C'].plot(figsize=(12,6), linestyle='--', color='black', legend='Open_C')

no_outlier_prices['Close_C'].plot(figsize=(12,6), linestyle='-', color='grey', legend='Close_C')

no_outlier_prices['Low_C'].plot(figsize=(12,6), linestyle=':', color='black', legend='Low_C')

no_outlier_prices['High_C'].plot(figsize=(12,6), linestyle='-.', color='grey', legend='High_C')

In [ ]:
pd.set_option('display.max_rows', 5)

In [ ]:
close_prices = no_outlier_prices[['Close_A', 'Close_B', 'Close_C']]

In [ ]:
delta_close_prices = (close_prices.shift(-1) - close_prices).fillna(0)

delta_close_prices.columns = ['Delta_Close_A', 'Delta_Close_B', 'Delta_Close_C']

delta_close_prices

In [ ]:
pd.set_option('display.max_rows', None)

delta_close_prices.describe()

In [ ]:
# histogram plot
delta_close_prices['Delta_Close_A'].plot(kind='hist', bins=100, figsize=(12,6), color='black', grid=True)

In [ ]:
# box plot
delta_close_prices['Delta_Close_B'].plot(kind='box', figsize=(12,6), color='black', grid=True)

In [ ]:
# Correlation chart
pd.plotting.scatter_matrix(delta_close_prices, figsize=(10,10), color='black', alpha=0.75, diagonal='kde', grid=True)

In [ ]:
plt.figure(figsize=(6,6))

sn.heatmap(delta_close_prices.corr(), annot=True, square=True, linewidths=2)

A (DJIA) and B (SPY) have very strong positive correlations, which makes sense since both are large cap equity indices. C (VIX) has strong negative correlations with both A (DJIA) and B (SPY), which also makes sense since during periods of prosperity, volatility remains low and markets rise, and during periods of crisis, volatility spikes and markets drop.

In [ ]:
import dtale

dtale.show(valid_close_df)

金融交易策略采用各种不同的订单类型，其中一些最常见的包括市价订单、带价格保护的市价订单、立即取消（IOC）订单、填写和取消（FAK）订单、有效至当天（GTD）订单、长效（GTC）订单、止损订单和冰山订单。我们将专注于市价订单、IOC 和 GTC。

市价订单
市价订单是需要立即以当前市场价格执行的买入或卖出订单，当执行的即时性优于执行价格时使用。

这些订单将以订单价格执行对立方的所有可用订单，直到要求的所有数量被执行。如果没有可用的流动性可以匹配，它可以被配置为停留在订单簿中或到期。停留在订单簿中意味着订单变为待定订单，被添加到订单簿中供其他参与者进行交易。到期意味着剩余订单数量被取消，而不是被添加到订单簿中，因此新订单无法与剩余数量匹配。

因此，例如，买入市价订单将与订单簿中从最佳价格到最差价格的所有卖出订单匹配，直到整个市价订单被执行。

这些订单可能会遭受极端的滑点，滑点被定义为已执行订单价格与发送订单时市场价格之间的差异。

IOC 订单
IOC 订单无法以比发送价格更差的价格执行，这意味着买入订单无法以高于订单价格的价格执行，卖出订单也无法以低于订单价格的价格执行。这个概念被称为限价，因为价格受限于订单可以执行的最差价格。

IOC 订单将继续与订单方的订单进行匹配，直到出现以下情况之一：

IOC 订单的全部数量被执行。

对方的被动订单价格比 IOC 订单的价格差。

IOC 订单部分执行，剩余数量到期。

如果 IOC 订单的价格优于另一方的最佳可用订单（即，买单低于最佳卖出价，或卖单高于最佳买入价），则根本不会执行，而只会过期。

GTC 订单可以无限期存在，并需要特定的取消订单。

限价订单簿
交易所接受来自所有市场参与者的订单请求，并将其保存在限价订单簿中。限价订单簿是交易所在任何时间点上所有可见订单的视图。

买单（或竞价）按照从最高价格（即，最佳价格）到最低价格（即，最差价格）的顺序排列，而卖单（即卖出或报价）则按照从最低价格（即，最佳价格）到最高价格（即，最低价格）的顺序排列。

最高竞价价格被认为是最佳竞价价格，因为具有最高买价的买单首先被匹配，而对于卖价，情况相反，即具有最低卖价的卖单首先匹配。

相同方向、相同价格水平的订单按照先进先出（FIFO）的顺序排列，也被称为优先顺序 - 优先级更高的订单排在优先级较低的订单前面，因为优先级更高的订单比其他订单先到达了交易所。其他条件相同（即，订单方向、价格和数量相同）的情况下，优先级更高的订单将在优先级较低的订单之前执行。

交易所撮合引擎
电子交易所的撮合引擎使用交易所撮合算法执行订单的撮合。撮合过程包括检查市场参与者输入的所有活跃订单，并将价格交叉的订单进行匹配，直到没有可以匹配的未匹配订单为止 - 因此，价格在或高于其他卖单的买单与之匹配，反之亦然，即价格在或低于其他买单的卖单与之匹配。剩余订单保留在交易所撮合簿中，直到新的订单流入，如果可能的话，进行新的匹配。

在 FIFO 匹配算法中，订单首先按照价格从最佳价格到最差价格进行匹配。因此，来自最佳价格的买单会尝试与摆放在最低价格到最高价格的卖单（即要价/出价）匹配，而来自最高价格的卖单会尝试与摆放在最高价格到最低价格的买单匹配。新到达的订单将根据特定的规则进行匹配。对于具有更好价格的主动订单（价格优于另一侧的最佳价格水平的订单），它们将按照先到先服务的原则进行匹配，即首先出现的订单会提取流动性，因此首先匹配。对于坐在订单簿中的被动挂单，因为它们不会立即执行，它们将根据先到先服务的优先级进行分配。这意味着同一方和相同价格的订单将根据它们到达匹配引擎的时间进行排列；时间较早的订单将获得更好的优先级，因此有资格首先匹配。

算法交易系统研究框架包括回测、交易后分析（PTA）和信号研究组件等组件。

In [ ]:
valid_close_complete['Open_A'].plot(figsize=(12,6), linestyle='--', color='black', legend='Open_A')
valid_close_complete['Close_A'].plot(figsize=(12,6), linestyle='-', color='grey', legend='Close_A')
valid_close_complete['Low_A'].plot(figsize=(12,6), linestyle=':', color='black', legend='Low_A')
valid_close_complete['High_A'].plot(figsize=(12,6), linestyle='-.', color='grey', legend='High_A')

In [ ]:
valid_close_complete['Open_C'].plot(figsize=(12,6), linestyle='--', color='black', legend='Open_C')
valid_close_complete['Close_C'].plot(figsize=(12,6), linestyle='-', color='grey', legend='Close_C')
valid_close_complete['Low_C'].plot(figsize=(12,6), linestyle=':', color='black', legend='Low_C')
valid_close_complete['High_C'].plot(figsize=(12,6), linestyle='-.', color='grey', legend='High_C')

In [ ]:
no_outlier_prices = prices_only[(np.abs(stats.zscore(prices_only)) < 6).all(axis=1)

In [ ]:
pd.plotting.scatter_matrix(delta_close_prices, figsize=(10,10), color='black', alpha=0.75, diagonal='kde', grid=True)